In [10]:
import pandas as pd

from modifiedGlicko import Team, ratingUpdate
import math
# import glicko2

init_d = 75
init_v = 0.001

In [11]:
df = pd.read_csv("bucs_results.csv")

postcodes = pd.read_csv("institute_postcodes.csv")

# print the first few rows of the DataFrame to verify that it was loaded correctly
df.head()
df = df.drop(columns=['No Score Type', 'Walkover Reason','Knockout Fixtures', 'Number of Walkovers', 'Voids', 'Blowout Game(s)', 'Zero Score?', 'Blowout to Zero', 'Team Conceding Walkover', 'Fixtures',
       'League Fixtures', 'Time'])

In [12]:
df = df[df["Fixture type"]=="League"]

In [13]:
def generate_glicko_init_rating(comp):
    if "National" in comp:
        return 3000
    if "North/South" in comp or "(North)" in comp or "(South)" in comp or "Premier" in comp:
        return 2300
    if "Tier 1" in comp:
        return 2000
    if "Tier 2" in comp:
        return 1500
    if "Tier 3" in comp:
        return 1200
    if "Tier 4" in comp:
        return 900
    if "Tier 5" in comp:
        return 700

In [14]:
df['HG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)
df['AG_RANK_Initial'] = df.apply(lambda row: generate_glicko_init_rating(row.Competition), axis=1)

In [15]:
team_gscores = {}

for i, team in df[df["Community"]=="BUCS Hockey 22-23"].iterrows():
# for i, team in df.iterrows():
    team_gscores[team.Home] = team.HG_RANK_Initial
    team_gscores[team.Away] = team.AG_RANK_Initial
    if math.isnan(team.HG_RANK_Initial):
        print(team.Home, team.HG_RANK_Initial, "Home", team.Competition)
    if math.isnan(team.AG_RANK_Initial):
        print(team.Away, team.AG_RANK_Initial, "Away", team.Competition)
    
#     print(team_gscores[team.Home], team_gscores[team.Away])
    
# print(team_gscores)

team_gObj = {}

for team, gscore in team_gscores.items():
    team_gObj[team] = Team(team, gscore, initial_deviation=init_d, initial_volatility=init_v)
    
# del team_gscores 

In [16]:
for i, row in df.iterrows():
    row['Home'] = row['Home'].replace("   ", " ")
    row['Away'] = row['Away'].replace("   ", " ")
    row['Home'] = row['Home'].replace("  ", " ")
    row['Away'] = row['Away'].replace("  ", " ")
    if row['Home'] not in team_gObj:
        team_gObj[row['Home']] = Team(row['Home'], row['HG_RANK_Initial'], initial_deviation=init_d, initial_volatility=init_v)
    home_gobj = team_gObj[row['Home']]
    
    if row['Away'] not in team_gObj:
        team_gObj[row['Away']] = Team(row['Away'], row['AG_RANK_Initial'], initial_deviation=init_d, initial_volatility=init_v)
    away_gobj = team_gObj[row['Away']]

    score_diff = int(row['Home Score']) - int(row['Away Score'])
 
    homeTeam, awayTeam = ratingUpdate(
        (home_gobj), 
        (away_gobj),
        score_diff
    )
    team_gObj[row['Home']] = homeTeam
    team_gObj[row['Away']] = awayTeam
    

# print(team_gObj)

In [17]:
for team, obj in team_gObj.items():
    if "Women's" in team:
        continue
    if "Nottingham M" not in team:
        continue
    
    print(team, obj.rating, obj.RD)

Nottingham Men's 1 3039.4485197811914 52.628932257248024
Nottingham Men's 7 912.1546227342383 66.601802310621
Nottingham Men's 2 2348.3914387134023 53.22096064052369
Nottingham Men's 6 1279.4007975891943 53.22121242231641
Nottingham Men's 5 1560.6140600205672 50.43104708298101
Nottingham Men's 4 2037.762951887401 50.4264344318252
Nottingham Men's 3 2029.7219041558874 50.430104701369594


In [18]:
output = {'names': [n for n in team_gObj.keys()], 'gs':[n.rating for n in team_gObj.values()], 'rd':[n.RD for n in team_gObj.values()]}
output = pd.DataFrame(output)
output.to_csv('team_ratings.csv', index=False)


